In [1]:
import numpy as np
# import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
import os
threads = os.cpu_count()
print(threads)

8


In [3]:
tf.config.threading.set_intra_op_parallelism_threads(threads)
tf.config.threading.set_inter_op_parallelism_threads(threads)

In [4]:
import pathlib
data_dir = "../dataset"
data_dir = pathlib.Path(data_dir).with_suffix('')

In [5]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

56286


In [6]:
batch_size = 32
img_height = 224
img_width = 224

In [7]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  # validation_split=0.0,
  # subset="training",
  seed=0,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 59990 files belonging to 2 classes.


In [8]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=0,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 59990 files belonging to 2 classes.
Using 11998 files for validation.


In [9]:
class_names = train_ds.class_names
print(class_names)

['false', 'true']


In [10]:
# AUTOTUNE = tf.data.AUTOTUNE

# train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [11]:
normalization_layer = layers.Rescaling(1./255)

In [12]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [13]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [14]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 64)        3

In [16]:
checkpointFilepath = 'model/model_{epoch:02d}_{val_accuracy:.2f}.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpointFilepath, 
    monitor='val_accuracy', 
    verbose=1, 
    save_best_only=True, 
    period=1
)

In [17]:
epochs=50
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[checkpoint]
)

Epoch 1/50


1875/1875 [==============================] - ETA: 0s - loss: 0.5134 - accuracy: 0.7923
Epoch 1: val_accuracy improved from -inf to 0.78896, saving model to model/model_01_0.79.h5
1875/1875 [==============================] - 2291s 1s/step - loss: 0.5134 - accuracy: 0.7923 - val_loss: 0.5089 - val_accuracy: 0.7890
Epoch 2/50


/home/aura/miniconda3/envs/DS/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1875/1875 [==============================] - ETA: 0s - loss: 0.5048 - accuracy: 0.7940
Epoch 2: val_accuracy improved from 0.78896 to 0.79172, saving model to model/model_02_0.79.h5
1875/1875 [==============================] - 2243s 1s/step - loss: 0.5048 - accuracy: 0.7940 - val_loss: 0.5040 - val_accuracy: 0.7917
Epoch 3/50
1875/1875 [==============================] - ETA: 0s - loss: 0.4977 - accuracy: 0.7954
Epoch 3: val_accuracy improved from 0.79172 to 0.79322, saving model to model/model_03_0.79.h5
1875/1875 [==============================] - 2206s 1s/step - loss: 0.4977 - accuracy: 0.7954 - val_loss: 0.4900 - val_accuracy: 0.7932
Epoch 4/50
1875/1875 [==============================] - ETA: 0s - loss: 0.4830 - accuracy: 0.7991
Epoch 4: val_accuracy improved from 0.79322 to 0.79738, saving model to model/model_04_0.80.h5
1875/1875 [==============================] - 2201s 1s/step - loss: 0.4830 - accuracy: 0.7991 - val_loss: 0.4794 - val_accuracy: 0.7974
Epoch 5/50
1875/1875 [=====

KeyboardInterrupt: 